In [11]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import os
import random
import wave
from array import array
import soundfile as sf
import sounddevice as sd
import scipy.io.wavfile as ww
from scipy.io.wavfile import read
import glob

In [57]:
PATH_NOISE= os.getcwd()+'/data/noise'
WORD_TO_TRAIN = ['ba','ta','si','mi','nu','ju']
PATH_TO_TRAIN = os.getcwd()+'/data/train'
PATH_TO_TEST = os.getcwd()+'/data/test'
FILE_PREFIX = "ns"
WORKPATH = PATH_TO_TRAIN


chunk = 2048
CHANNELS = 1
RATE = 16000
length_capture = 20  #20 ms
overlap_frame = 4    #4 ms
start_at = 7         #maybe not used
end_at = 100
DEBUG = False

In [8]:
def getlistwav(path, dir=""):
    if dir == "": dir = path + "/"
    tmp = os.listdir(path)
    result = []
    for i in tmp:
        if str(i).find('wav',0)>0: result.append(dir+i)
    return result

In [72]:
def writefile(filename, data):
    p = pyaudio.PyAudio()
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close() 

def addblank(filename, front=0, back=0):
    wv, sr = sf.read(filename)
    wv=np.pad(wv,(front,back),'constant')
    plt.plot(wv)
    return wv

def getlen(filename):
    wv, sr = sf.read(filename)
    print('Length:',len(wv))
    return len(wv)

def pitch(filename,target=None, n_steps=0.0):
    wv,sr = sf.read(filename)
    print(wv)
    wv = wv.T
    wv = librosa.resample(wv,sr,44100,scale=True)
    y_ = librosa.effects.pitch_shift(wv, 44100, n_steps=n_steps)
    y_ = librosa.resample(y_,44100,sr,scale=True)
    print(y_)
    sd.play(y_,samplerate=sr)
    if target!=None: sf.write(target,y_,sr)

def add_random_noise(filename, target):

    #add some blanks front and back
    front = random.randint(0,5000)
    back = random.randint(0,5000)

    wv, sr = sf.read(filename)
    wv=np.pad(wv,(front,back),'constant')


    #random pitch
    ptch = random.uniform(-3,3)

    wv = wv.T
    wv = librosa.resample(wv,sr,44100,scale=True)
    wv = librosa.effects.pitch_shift(wv, 44100, n_steps=ptch)
    #y_ = librosa.resample(y_,44100,sr,scale=True)
    #print(y_)

    #get the noise
    lnoise = getlistwav(PATH_NOISE, dir="",)
    random.shuffle(lnoise)
    wv2, sr2 = sf.read(lnoise[0])
    

    wv2 = wv2.T
    wv2 = librosa.resample(wv2,sr2,44100,scale=True)
    
    rv = random.randint(0,len(wv2)-len(wv))
    wv2 = wv2[rv:rv+len(wv),]
    #wv3 = ((wv * random.uniform(0.9,1.3)) + (wv2 * random.uniform(0,0.4))) / 2
    wv3 = ( (wv * 3) + (wv2 * 2) ) / 6

    wv3 = librosa.resample(wv3,44100,sr,scale=True)

    #sd.play(wv3,samplerate=sr)
    if target!=None: sf.write(target,wv3,sr)


In [73]:
class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

In [71]:
def gennoise(WORKPATH=WORKPATH,totalfile=1):
    """ 
    Gunakan generate ini untuk membuat SET dari yang ingin di training atau
    untuk SET TESTING, gunakan frame untuk menandakan panjang dari frame
    yang ingin di deteksi
    """
    files=[]
    #Generate AUDIO FILE

    #create the dir 
    for i in WORD_TO_TRAIN:
        try:
            os.mkdir(PATH_TO_TRAIN+"/"+i)
            os.mkdir(PATH_TO_TEST+"/"+i)
        except:
            pass
        
    
    for i in WORD_TO_TRAIN:
        #gunakan PATH_TO_TEST atau PATH_TO_TRAIN
        workdir = WORKPATH+"\\"+i+"\\"
        filestonoise = glob.glob(workdir+"*.wav")
        for tn in range(totalfile):
            for f in filestonoise:
                rann = str(random.randint(0,5000))
                t_file=f.split('.wav')[0]+'-'+FILE_PREFIX+"-"+rann+".wav"
                
                add_random_noise(f,t_file)
                print(t_file)
    
            print("Selesai proses merekam ke:",workdir)
    return files


In [70]:
gennoise(WORKPATH,totalfile=2)

16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-2741-ns-2404.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-3058-ns-971.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-3854-ns-538.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-4191-ns-2602.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-678-ns-396.wav
Selesai proses merekam ke: E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-2741-ns-4811.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-3058-ns-2065.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-3854-ns-564.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-4191-ns-2126.wav
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\ba-irza-678-ns-834.wav
Selesai proses merekam ke: E:\data\python\3.6\Keras-Ayukngaji/data/train\ba\
16000
E:\data\python\3.6\Keras-Ayukngaji/data/train\t

[]

In [59]:
import webrtcvad as wrt
a = wrt.Vad()
fg = frame_generator(20,wv,16000)

fg = list(fg)

lb = []
for ff in fg:
    lb.append(a.is_speech(ff.bytes,16000))


In [63]:
len(fg[1].bytes)

640

In [54]:
ipd.Audio("record.wav")

In [17]:
!python example.py 3 "record.wav"

11101111111111+(0.12)111111111001110000000000-(1.1400000000000008) Writing chunk-00.wav
00000000


In [10]:
add_random_noise("chunk-00.wav", "record.wav")

16000


In [40]:
import webrtcvad as wtc
a = wtc.Vad()
wv, sr = sf.read('record.wav')

In [41]:
a.is_speech

In [18]:
for i in range(10):
    add_random_noise('mytest-16000.wav','mi.wav')

16000
16000
16000
16000
16000
16000
16000
16000
16000
16000


In [5]:
add_random_noise('record.wav','jack.wav')

NameError: name 'add_random_noise' is not defined